In [1]:
from google.colab import files
uploaded = files.upload()

Saving dataPreprocessed.csv to dataPreprocessed.csv


In [2]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
import pandas as pd
import io
import spacy
from gensim.models.word2vec import Word2Vec
import re, string
from nltk.corpus import stopwords

stopword = stopwords.words('english')

df = pd.read_csv(io.StringIO(uploaded['dataPreprocessed.csv'].decode('utf-8')))
df


,X,Title,Company,Location,Type,Salary,Contract_type,Job_Description,JobType,Industry,...,exp_normailzed,type_norm,Contract_type_norm,salary1,salary_min,salary_max,salary_avg,salary_norm,label,annual_pay
0,1,Construction Superintendent,"Curtis Construction Company, Inc.","Stoughton, MA 02072",Onsite,"$60,000 - $130,000",Full-time,Greater Boston based General Contractor is see...,Construction Management,Civil Engineering,...,0.5,-0.063628,1,"$60,000 - $130,000",60000.0,130000.0,95000.0,95000.0,Medium,95000.0
1,2,Regional Construction Manager,Chipotle Mexican Grill,"Atlanta, GA 30309",Onsite,"$89,000 - $161,000",Contract,Regional Construction Manager\nDescription\n\n...,Construction Management,Civil Engineering,...,0.1,-0.063628,0,"$89,000 - $161,000",89000.0,161000.0,125000.0,125000.0,High,125000.0
2,3,Construction Project Manager,Perfetto Contracting Co Inc,"Brooklyn, NY",Onsite,"$170,000 - $200,000",Full-time,Innovating infrastructure one project at a tim...,Construction Management,Civil Engineering,...,2.5,-0.063628,1,"$170,000 - $200,000",170000.0,200000.0,185000.0,185000.0,High,185000.0
3,5,Construction Manager,Chipotle Mexican Grill,"Denver, CO 80210",Onsite,"$89,000 - $161,000",Contract,Construction Manager\nDescription\n\nCULTIVATI...,Construction Management,Civil Engineering,...,0.1,-0.063628,0,"$89,000 - $161,000",89000.0,161000.0,125000.0,125000.0,High,125000.0
4,8,Construction Manager,Black & Veatch Corporation,"San Diego, CA",Onsite,"$178,000 - $220,000",Full-time,"Together, we own our company, our future, and ...",Construction Management,Civil Engineering,...,10.0,-0.063628,1,"$178,000 - $220,000",178000.0,220000.0,199000.0,199000.0,High,199000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3764,873,Up Scale Server- Daytime,Queen Mary Tea,"2912 NE 55th St, Seattle, WA 98105",Onsite,$30 - $45,Contract,"Established and beautiful, high end, restauran...",Restaurant_server,Service,...,0.1,-0.063628,0,$30 - $45,30.0,45.0,37.5,78000.0,Medium,78000.0
3765,874,Bartender and Server,The Spice Sea,"3345 W Greenway Rd, Phoenix, AZ 85053",Onsite,$19 - $30,Contract,"no experience, no problem. We will teach.\nJob...",Restaurant_server,Service,...,NaN,-0.063628,0,$19 - $30,19.0,30.0,24.5,50960.0,Low,50960.0
3766,875,Server - Average pay $30/hour - Weekly Pay!,Pisco y Nazca Ceviche Gastrobar,"1823 L Street NW, Washington, DC 20036",Onsite,$28 - $32,Contract,Pisco Y Nazca Ceviche Gastrobar (DC) is HIRING...,Restaurant_server,Service,...,0.1,-0.063628,0,$28 - $32,28.0,32.0,30.0,62400.0,Low,62400.0
3767,879,Fine Dining Server,Heirloom Bistro,"7713 N Kings Hwy, Myrtle Beach, SC 29572",Onsite,"$30,000 - $76,243",Contract,Servers Needed Immediately For Brand New Fine ...,Restaurant_server,Service,...,0.3,-0.063628,0,"$30,000 - $76,243",30000.0,76243.0,53121.5,53121.5,Low,53121.5


In [6]:
class_df = df.groupby('label').count()
class_df.style.background_gradient(cmap='PiYG')

,X,Title,Company,Location,Type,Salary,Contract_type,Job_Description,JobType,Industry,experience,exp_normailzed,type_norm,Contract_type_norm,salary1,salary_min,salary_max,salary_avg,salary_norm,annual_pay
label,,,,,,,,,,,,,,,,,,,,
High,1072,1072,1072,1072,1072,1072,1072,546,1072,1072,342,342,1072,1072,1072,1072,1072,1072,1072,1072
Low,750,750,750,750,750,750,750,578,750,750,292,292,750,750,750,750,750,750,750,750
Medium,1947,1947,1933,1947,1947,1947,1947,1134,1947,1947,704,704,1947,1947,1947,1947,1947,1947,1947,1947


In [7]:
def clean(text):
    text = text.lower() # lower case
    text = re.sub(r'https?://\S+|www\.\S+', " ", text)#URL
    text = re.sub(r'@\w+',' ',text) # mentions
    text = re.sub(r'#\w+', ' ', text) #hashtags
    text= re.sub(r'[^\w\s]',' ',text) #punctuations
    text= re.sub(r'\d+', ' ', text) #digits
    text = re.sub('\[.*?\]',' ', text) #any punctuations left
    text = re.sub("[^a-z0-9]"," ", text)# any others charcters other than numbers and letters
    text = text.split() #stopwords
    text = " ".join([word for word in text if not word in stopword])
    return text

df['Job_Description'] = df['Job_Description'].astype(str).apply(lambda x: clean(x))

In [8]:
df.head(10)

,X,Title,Company,Location,Type,Salary,Contract_type,Job_Description,JobType,Industry,...,exp_normailzed,type_norm,Contract_type_norm,salary1,salary_min,salary_max,salary_avg,salary_norm,label,annual_pay
0,1,Construction Superintendent,"Curtis Construction Company, Inc.","Stoughton, MA 02072",Onsite,"$60,000 - $130,000",Full-time,greater boston based general contractor seekin...,Construction Management,Civil Engineering,...,0.5,-0.063628,1,"$60,000 - $130,000",60000.0,130000.0,95000.0,95000.0,Medium,95000.0
1,2,Regional Construction Manager,Chipotle Mexican Grill,"Atlanta, GA 30309",Onsite,"$89,000 - $161,000",Contract,regional construction manager description cult...,Construction Management,Civil Engineering,...,0.1,-0.063628,0,"$89,000 - $161,000",89000.0,161000.0,125000.0,125000.0,High,125000.0
2,3,Construction Project Manager,Perfetto Contracting Co Inc,"Brooklyn, NY",Onsite,"$170,000 - $200,000",Full-time,innovating infrastructure one project time per...,Construction Management,Civil Engineering,...,2.5,-0.063628,1,"$170,000 - $200,000",170000.0,200000.0,185000.0,185000.0,High,185000.0
3,5,Construction Manager,Chipotle Mexican Grill,"Denver, CO 80210",Onsite,"$89,000 - $161,000",Contract,construction manager description cultivating b...,Construction Management,Civil Engineering,...,0.1,-0.063628,0,"$89,000 - $161,000",89000.0,161000.0,125000.0,125000.0,High,125000.0
4,8,Construction Manager,Black & Veatch Corporation,"San Diego, CA",Onsite,"$178,000 - $220,000",Full-time,together company future shared success employe...,Construction Management,Civil Engineering,...,10.0,-0.063628,1,"$178,000 - $220,000",178000.0,220000.0,199000.0,199000.0,High,199000.0
5,12,Construction Sales - Project Management,The Neill Company,"Orange Park, FL 32073",Onsite,"$65,000 - $135,000",Contract,neill company premier commercial residential c...,Construction Management,Civil Engineering,...,1.5,-0.063628,0,"$65,000 - $135,000",65000.0,135000.0,100000.0,100000.0,Medium,100000.0
6,16,Residential Construction Project Manager,RTG,"San Jose, CA",Onsite,"$125,000 - $150,000",Full-time,well established growing construction company ...,Construction Management,Civil Engineering,...,NaN,-0.063628,1,"$125,000 - $150,000",125000.0,150000.0,137500.0,137500.0,High,137500.0
7,18,Experienced Construction Management Specialist,BOEING,"Springfield, VA",Onsite,"$93,500 - $126,500",Contract,boeing innovate collaborate make world better ...,Construction Management,Civil Engineering,...,0.1,-0.063628,0,"$93,500 - $126,500",93500.0,126500.0,110000.0,110000.0,Medium,110000.0
8,19,"Residential Property/Construction Manager, Gen...",Soar Real Estate,"Fort Lee, NJ",Onsite,"$140,000 - $190,000",Full-time,residential property manager general manager c...,Construction Management,Civil Engineering,...,0.3,-0.063628,1,"$140,000 - $190,000",140000.0,190000.0,165000.0,165000.0,High,165000.0
9,20,Construction Project Manager,CRB,"12625 High Bluff Dr, San Diego, CA 92130",Onsite,"$131,700 - $210,700",Full-time,company description crb nearly expert professi...,Construction Management,Civil Engineering,...,NaN,-0.063628,1,"$131,700 - $210,700",131700.0,210700.0,171200.0,171200.0,High,171200.0


In [9]:
import spacy

# spacy.load is a method in the Spacy library for loading pre-trained models for natural language processing.
# load english language model and create nlp object from it
# Pre-trained English language model called en_core_web_sm. 
# This model is a small and fast model for English language text processing, and it includes features such as part-of-speech tagging, named entity recognition, and dependency parsing
nlp = spacy.load("en_core_web_sm") 

def preprocess(text):
    #For converting paragrapg to sentence
    # spacy. is object oriented
    # Dr. Ram is eating.He is not available.
    # -> [Dr. Ram is eating. He is not available]
    doc = nlp(text)

    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    
    return " ".join(filtered_tokens) 

In [10]:
df['preprocessed_txt'] = df['Job_Description'].apply(preprocess) 

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    df.preprocessed_txt, 
    df.label, 
    test_size=0.2, # 20% samples will go to test dataset
    random_state=2022, #keep using the same sample for training or testing
    stratify=df.label #balance category used in training the model
)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer


In [13]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# create CountVectorizer object
vectorizer = CountVectorizer(ngram_range= (1,5)).fit(x_train)

# fit_transform on training data
train_X_cv = vectorizer.fit_transform(x_train)
# transform test data
test_X_cv = vectorizer.transform(x_test)

# create TfidfTransformer object
tfidf_transformer = TfidfTransformer()

train_X_tfidf = tfidf_transformer.fit_transform(train_X_cv)
test_X_tfidf = tfidf_transformer.transform(test_X_cv)

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


In [15]:
lr = LogisticRegression(penalty="l2",multi_class = "multinomial",solver = "saga", C=1, max_iter=5000)
lr.fit(train_X_tfidf,y_train)
lr_pred = lr.predict(test_X_tfidf)
print(classification_report(y_test, lr_pred))

              precision    recall  f1-score   support

        High       0.90      0.25      0.39       214
         Low       0.91      0.47      0.62       150
      Medium       0.62      0.97      0.75       390

    accuracy                           0.67       754
   macro avg       0.81      0.56      0.59       754
weighted avg       0.75      0.67      0.62       754



In [32]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range= (1,3))), # convert text to a matrix of token counts
    ('clf', LogisticRegression(penalty="l2",multi_class = "multinomial",solver = "saga", C=1, max_iter=5000)), # train a Naive Bayes classifier
])
pipeline.fit(x_train, y_train)

predicted_label = pipeline.predict(x_test)
print(classification_report(y_test, predicted_label))

              precision    recall  f1-score   support

        High       0.84      0.39      0.54       214
         Low       0.81      0.55      0.66       150
      Medium       0.65      0.93      0.77       390

    accuracy                           0.70       754
   macro avg       0.77      0.62      0.65       754
weighted avg       0.74      0.70      0.68       754



In [33]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range= (1,5))), # convert text to a matrix of token counts
    ('clf', LogisticRegression(penalty="l2",multi_class = "multinomial",solver = "saga", C=1, max_iter=5000)), # train a Naive Bayes classifier
])
pipeline.fit(x_train, y_train)

predicted_label = pipeline.predict(x_test)
print(classification_report(y_test, predicted_label))

              precision    recall  f1-score   support

        High       0.85      0.38      0.53       214
         Low       0.81      0.55      0.65       150
      Medium       0.65      0.93      0.77       390

    accuracy                           0.70       754
   macro avg       0.77      0.62      0.65       754
weighted avg       0.74      0.70      0.68       754



In [29]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range= (1,5))), # convert text to a matrix of token counts
    ('tfidf', TfidfTransformer()), # apply TF-IDF normalization to the matrix
    ('clf', LogisticRegression(penalty="l2",multi_class = "multinomial",solver = "saga", C=1, max_iter=5000)), # train a Naive Bayes classifier
])
pipeline.fit(x_train, y_train)

predicted_label = pipeline.predict(x_test)
print(classification_report(y_test, predicted_label))

              precision    recall  f1-score   support

        High       0.90      0.25      0.39       214
         Low       0.91      0.47      0.62       150
      Medium       0.62      0.97      0.75       390

    accuracy                           0.67       754
   macro avg       0.81      0.56      0.59       754
weighted avg       0.75      0.67      0.62       754



In [28]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range= (1,3))), # convert text to a matrix of token counts
    ('tfidf', TfidfTransformer()), # apply TF-IDF normalization to the matrix
    ('clf', MultinomialNB()), # train a Naive Bayes classifier
])
pipeline.fit(x_train, y_train)

predicted_label = pipeline.predict(x_test)
print(classification_report(y_test, predicted_label))

              precision    recall  f1-score   support

        High       1.00      0.02      0.04       214
         Low       1.00      0.12      0.21       150
      Medium       0.53      1.00      0.70       390

    accuracy                           0.55       754
   macro avg       0.84      0.38      0.32       754
weighted avg       0.76      0.55      0.41       754



In [38]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range= (1,5))), # convert text to a matrix of token counts
    ('clf', MultinomialNB()), # train a Naive Bayes classifier
])
pipeline.fit(x_train, y_train)

predicted_label = pipeline.predict(x_test)
print(classification_report(y_test, predicted_label))

              precision    recall  f1-score   support

        High       0.85      0.32      0.46       214
         Low       0.91      0.46      0.61       150
      Medium       0.62      0.96      0.76       390

    accuracy                           0.68       754
   macro avg       0.79      0.58      0.61       754
weighted avg       0.74      0.68      0.64       754

